# Tools

## 1.- Setup inicial

### 1.1- Instalar librerías 

In [ ]:
# Instalar librerías
#!pip install openai
#!pip install spacy
#!pip install tenacity
#!pip install python-dotenv
#!spacy download es_core_news_sm

### 1.2.- Cargar librerías

In [ ]:
import os
import openai
import difflib
import spacy
import datetime
from dateutil import parser
from dotenv import load_dotenv
from IPython.display import display, HTML
from tenacity import retry, wait_random_exponential, stop_after_attempt

### 1.3.- OpenAI API

In [ ]:
# Cargar secretos y configuración desde el archivo .env
load_dotenv()

# Configurar la clave de la API de OpenAI
openai.api_key = os.getenv("OPENAI_API_KEY")
embedding_model = os.getenv("OPENAI_EMBEDDING_MODEL")
print("OpenAI API key: {}".format(openai.api_key[:5] + '...' + openai.api_key[-5:]))

# Nombres de los modelos
gpt35_model = os.getenv("OPENAI_GPT35_MODEL")
gpt35_16k_model = os.getenv("OPENAI_GPT35_16K_MODEL")
gpt4_model = os.getenv("OPENAI_GPT4_MODEL")
print("GPT-3.5-Turbo model: {}".format(gpt35_model))
print("GPT-3.5-Turbo-16k model: {}".format(gpt35_16k_model))
print("GPT-4 model: {}".format(gpt4_model))

### 1.4.- Importar clase Tools

In [ ]:
from agents.tools import Tool, Parameter

### 1.5.- Definir funciones útiles

In [ ]:
@retry(wait=wait_random_exponential(min=1, max=20), stop=stop_after_attempt(3))
def generate_text(prompt, model=gpt35_model, messages=[], max_tokens=200, temperature=1.0, top_p=1.0, frequency_penalty=0.0, presence_penalty=0.0, stop=None):
    _messages = []
    _messages.extend(messages)
    _messages.append({"role": "user", "content": prompt})

    response = openai.ChatCompletion.create(
        model=model,
        messages=_messages,
        max_tokens=max_tokens,
        temperature=temperature,
        top_p=top_p,
        frequency_penalty=frequency_penalty,
        presence_penalty=presence_penalty,
        stop=stop
    )
    return response["choices"][0]["message"]["content"]

In [ ]:
def identify_cities(text):
    nlp = spacy.load("es_core_news_sm")  # Carga el modelo de procesamiento de texto en español
    doc = nlp(text)
    
    # Encuentra nombres de ciudades en el texto
    cities = [entity.text for entity in doc.ents if entity.label_ == "LOC"]
    return cities

## 2.- Ejemplos

### 2.1 Suma de números o concatenación de strings

#### 2.1.1 Definición de funciones

In [ ]:
def sumar(a: int, b: int) -> int:
    return a + b

def concatenar_strings(s1: str, s2: str) -> str:
    return s1 + s2

#### 2.1.2 Instanciar funciones como Tools

In [ ]:
sumar_tool = Tool(
    name="Sumar números",
    func=sumar,
    description="Esta función recibe dos números enteros como entrada y devuelve la suma de estos dos números.",
    arguments=[
        Parameter(name="a", description="Primer número", type=int, required=True),
        Parameter(name="b", description="Segundo número", type=int, required=True)
    ]
)

concatenar_tool = Tool(
    name="Concatenar strings",
    func=concatenar_strings,
    description="Esta función toma dos palabras como entrada y las combina en una sola cadena de texto, sin agregar espacios ni caracteres adicionales.",
    arguments=[
        Parameter(name="s1", description="Primera cadena", type=str, required=True),
        Parameter(name="s2", description="Segunda cadena", type=str, required=True)
    ]
)

#Definir lista de tools
tools1 = [sumar_tool, concatenar_tool]

#### 2.1.3 Función de decisión

In [ ]:
def LLM_decision1(user_input: str, tools: list[Tool]) -> Tool:
    # Basado en el input del usuario, determina qué herramienta usar.
    # (Aquí estamos simplificando y usando solo las descripciones, pero en la realidad podría ser más complejo)
    best_match = None
    highest_similarity = 0
    for tool in tools:
        similarity = difflib.SequenceMatcher(None, user_input, tool.description).ratio()  # Función hipotética que compara similitud.
        if similarity > highest_similarity:
            highest_similarity = similarity
            best_match = tool
            
    if best_match == sumar_tool:
        numbers = [int(word) for word in user_input.split() if word.isdigit()]
        return best_match, {"a": numbers[0], "b": numbers[1]}
    else:
        # Supongamos que las cadenas a concatenar están entre comillas
        strings = [word.strip('"') for word in user_input.split() if word.startswith('"') and word.endswith('"')]
        return concatenar_tool, {"s1": strings[0], "s2": strings[1]}

#### 2.1.4 Ejemplos de uso

##### Sumar números

In [ ]:
input1 = "Sumar los números 5 y 7"

In [ ]:
tool1, args1 = LLM_decision1(input, tools)
display(HTML(f"<h5>Se ha seleccionado la función: {tool1.name}, sus argumentos son: {' y '.join(map(str, args1.values()))} </h5>"))

Se ejecuta la función

In [ ]:
result_q1 = tool1.execute(**args1)
display(HTML(f"<h5>Debería imprimir 12. El resultado obtenido es: {result_q1} </h5>"))  # Debería imprimir 12

---------------------------------------------------------------------------------------------------------------------------------------------------------

##### Sumar strings

In [ ]:
input2 = 'Genera un texto con estas palabras: "hola" y "mundo"'

In [ ]:
tool2, args2 = LLM_decision1(question2, tools)
display(HTML(f"<h5>Se ha seleccionado la función: {tool2.name}, sus argumentos son: {' y '.join(map(str, args2.values()))} </h5>"))

In [ ]:
result_q2 = tool2.execute(**args2)
display(HTML(f"<h5>Debería imprimir holamundo. El resultado obtenido es: {result_q2} </h5>"))  # Debería imprimir holamundo

### 2.2 Tiempo meteorológico o tiempo de viaje

#### 2.1.1 Definición de funciones

In [ ]:
import requests
from llm import generate_text

# Una función que consulta una API meteorológica
def consultar_clima(ciudad: str, fecha: int) -> dict:
    prompt = f"Cómo estará el clima en {ciudad} el {fecha}"
    response = generate_text(prompt, model=gpt4_model)
    return response

def tiempo_de_viaje(ciudad1: str, ciudad2: str) -> str:
    prompt = f"Cúal es el tiempo de viajes desde {ciudad1} a {ciudad2}"
    response = generate_text(prompt, model=gpt4_model)
    return response

#### 2.2.2 Instanciar funciones como Tools

In [ ]:
# Crea instancias de las funciones como objetos Tool
clima_tool = Tool(
    name = "Consultar clima", 
    func = consultar_clima, 
    description = "Esta función proporciona información simulada sobre las condiciones climáticas en una ciudad específica en una fecha determinada. Es útil para obtener una idea general del clima en un lugar en un momento particular y puede ser utilizado para planificar actividades al aire libre.",
    arguments = [
        Parameter("ciudad", str, "Nombre de la ciudad", True), 
        Parameter("fecha", datetime, "Fecha de consulta", True)
    ]
)


viaje_tool = Tool(
    name = "Tiempo de viaje", 
    func = tiempo_de_viaje, 
    description = "Esta función calcula el tiempo estimado de viaje entre dos ciudades. Utiliza datos simulados para proporcionar una estimación del tiempo que tomará llegar de una ciudad de origen a una ciudad de destino. Esta estimación puede ser útil para planificar tus viajes y actividades.", 
    arguments = [
        Parameter("ciudad1", str, "Ciudad de origen", True), 
        Parameter("ciudad2", str, "Ciudad de destino", True)
    ]
)

tools2 = [clima_tool, viaje_tool]

#### 2.2.3 Función de decisión

In [ ]:
def LLM_decision2(user_input: str, tools: list[Tool]) -> Tool:
    # Basado en el input del usuario, determina qué herramienta usar.
    # (Aquí estamos simplificando y usando solo las descripciones, pero en la realidad podría ser más complejo)
    best_match = None
    highest_similarity = 0
    for tool in tools:
        similarity = difflib.SequenceMatcher(None, user_input, tool.description).ratio()  # Función hipotética que compara similitud.
        if similarity > highest_similarity:
            highest_similarity = similarity
            best_match = tool
    if best_match == clima_tool:
        cities = identify_cities(user_input)
        date = parser.parse(user_input, fuzzy_with_tokens=True)
        return best_match, {"ciudad": cities[0], "fecha": date[0]}
    else:
        cities = identify_cities(user_input)
        return best_match, {"ciudad1": cities[0], "ciudad2": cities[1]}

#### 2.2.4 Ejemplos de uso

#### a. Tiempo meteorológico

In [ ]:
input3 = "Tiempo para el miércoles 25 de octubre en Barcelona"

Se aplica la función de decisión

In [ ]:
tool3, args3 = LLM_decision(input3, tools2)
display(HTML(f"<h5>Se ha seleccionado la función: {tool3.name}, sus argumentos son: {' y '.join(map(str, args3.values()))} </h5>"))

Se ejecuta la función y se verifica el resultado

In [ ]:
result_q3 = tool3.execute(args3)
display(HTML(f"<h5>Debería imprimir 12. El resultado obtenido es: {result_q3} </h5>"))  # Debería imprimir

#### b. Tiempo de viaje

In [ ]:
input4 = 'Tiempo de viaje de Madrid a Barcelona'

Se aplica la función de decisión

In [ ]:
tool4, args4 = LLM_decision(input4, tools)
display(HTML(f"<h5>Se ha seleccionado la función: {tool4.name}, sus argumentos son: {' y '.join(map(str, args4.values()))} </h5>"))

Se ejecuta la función y se verifica el resultado

In [ ]:
result_q4 = tool4.execute(**args4)
display(HTML(f"<h5>Debería imprimir holamundo. El resultado obtenido es: {result_q2} </h5>"))  # Debería imprimir 